# Power Outages
* **See the main project notebook for instructions to be sure you satisfy the rubric!**
* See Project 03 for information on the dataset.
* A few example prediction questions to pursue are listed below. However, don't limit yourself to them!
    * Predict the severity (number of customers, duration, or demand loss) of a major power outage.
    * Predict the cause of a major power outage.
    * Predict the number and/or severity of major power outages in the year 2020.
    * Predict the electricity consumption of an area.

Be careful to justify what information you would know at the "time of prediction" and train your model using only those features.

# Summary of Findings


### Introduction
In this project, we will look at the major power outage data in the continental U.S. from January 2000 to July 2016. After data cleaning, the dataset has 1534 observations (each occurence of outage), and 56 columns (location, cause, influential factors, time-stamp, aftermath of the outage etc.)

The classification problem is that we want to predict the **cause category** (target variable) for each major power outages given the year, the state, outage duration, population of the state, number of customers affected for each state (features). And we'll use **Recall** as our evaluation metric to examine our model, and make further improvements based on the value. The justification for this metric over others will be given in Baseline model section.

### Baseline Model
Our baseline model uses the Decision Tree Classifier, since the `CAUSE.CATEGORY` consists of 7 categories,and including 5 features: [`'YEAR'`, `'U.S._STATE'`, `'OUTAGE.DURATION'`, `'POPULATION'`, `'CUSTOMERS.AFFECTED'`]. Of which, `YEAR` column is ordinal, `U.S. STATE` is nominal, and remaining three columns are quantitative variables.

The reason why we choose year as one of the features is that as time progresses, we guessed that cause category might also change correspondingly due to technological improvement. We chose state since the geography, demography, consumption varies among different states, which might have an effect on power outage. We chose outage duration, since the length of each outage can depend on the severity, which also closely linked to the cause category. The population is feasible since the population is a straight forward representation of the scale of the state, difference in population might also affect the cause. At last, if cause category belongs to a macro scale influence, naturally many customers will be affected, making customers affected another viable feature.

In addition,choosing a good evaluation metric is quite challenging,since the `CAUSE.CATEGORY` has 7 causes for the outage, each with very diverse amount (using value_counts), it's relatively hard to calculate recall, specifity, precision, and etc., only if we repeat them for each category (it will be much easier to work with since they are for binarized-categorical variable like the tumor/no tumor, or terrorists/non-terrorists example). To tackle this problem, we first define a variable `categories` that includes all causes, and then assign it for the keyword argument `labels` for the confusion matrix. To better visualize True Positive, False Positive, False Negative and True Negative, we used multi-labeled confusion matrix that displays all seven categories respectively in a more visually straightforward 2x2 matrices. Parallel to our expectation, the multi-label confusion matrix has matching True Positives as values on the diagonal line of the ordinary confusion matrix. Since there are 7 categories, and they have unequal proportion, we value True Positives over all observations that are actually labeled positive the most (unlike tumor problem, True Negative doesn't give much information in this case, recall is most helpful),

$$
{\rm Recall} =\frac{TP}{P} =\frac{TP}{TP + FN}
$$

which follows that we will use `Recall` as our evaluation metric. More specifically, we will take the mean of all 7 categories' `recall` as the evaluation metric using `recall_score`. However, our evaluation metric can be further improved since we consider the case where TP = 0 and TP + FN = 0 to get a recall of 0 (but if FN is non-zero, TP is 0, we also get 0, they don't mean the same thing since TP + FN = 0 suggests that there are no positive labels in the input data). 

After training our baseline model, it has a mean of 0.464 recall over all categories on testing dataset, which is quite decent considering we have 7 categories.

### Final Model
To achieve our final model, we attempted 2 feature engineering that would theoretically improve our evaluation metric. We first calculates the population density using:

$$
{\rm Population\ density} = \frac {POPPCT\_URBAN * POPDEN\_URBAN + (1 - POPPCT\_URBAN) * POPDEN\_RURAL}{100} 
$$

We think that it will be a very effective feature that could potentially increase our recall score, since population density can be an indicative measure for different causes. For example, major outages that occur within areas with higher population density might due to `system operability disruption`. The second is StandardScaler. It will transform our data such that its distribution will have a mean value 0 and standard deviation of 1. We applied this on other numeric columns. 

With these two additional features, we have increased our recall score to 0.593. Then, we performed Grid Search on this final pipeline to obtain the final model for our prediction. It gives that the best pipeline parameters are: {'dtc__max_depth': 5, 'dtc__min_samples_leaf': 2, 'dtc__min_samples_split': 17}.

The model can be further improved if Grid Search is able to optimize parameters based on recall score rather than accuracy.

### Fairness Evaluation
To assess the fairness and bias of our model, we specifically chose the column `POPPCT_URBAN` to construct our X and Y subsets and hypothesized that the urban population percentage doesn't affect our recall scores.

We will binarize our datasets into:

`1`: which has `POPPCT_URBAN` higher than the median urban population percentage;

`0`: which are lower than the median.

Our test statistic would be the **absolute difference** between the **recall score** of these two subsets. 

***Null Hypothesis***: Urban population percentage, either higher or lower than its' median, doesn't have an effect on our recall score.(i.e. the difference between the recall scores between two subsets are due to chance)

***Alternative Hypothesis***: The urban population percentage does influence our recall score on two subsets.

After running 1000 permutation tests, we obtained a p-value of 0.762, rejecting the null hypothesis that the urban population percentage does influence our recall score on two subsets. Suggesting that the fairness can be further improved.

# Code

We first import necessary packages.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import datetime
from IPython.display import display
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import utils
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

This section of the code processes and cleans the dataset.

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_excel('outage.xlsx', skiprows = 5)
df = df.drop(0).drop(['variables','OBS'], axis = 1).reset_index(drop=True)
df

<ipython-input-5-d79707f54248>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel('outage.xlsx', skiprows = 5)


,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,NaN,NaN,3060,NaN,70000.0,11.60,9.18,6.81,9.28,2332915,2114774,2113291,6562520,35.549073,32.225029,32.202431,2308736.0,276286.0,10673.0,2595696.0,88.944776,10.644005,0.411181,51268,47586,1.077376,1.6,4802,274182,1.751391,2.2,5348119.0,73.27,15.28,2279,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
1,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,vandalism,NaN,1,NaN,NaN,12.12,9.71,6.49,9.28,1586986,1807756,1887927,5284231,30.032487,34.210389,35.727564,2345860.0,284978.0,9898.0,2640737.0,88.833534,10.791609,0.374820,53499,49091,1.089792,1.9,5226,291955,1.790002,2.2,5457125.0,73.27,15.28,2279,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
2,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,heavy wind,NaN,3000,NaN,70000.0,10.87,8.19,6.07,8.15,1467293,1801683,1951295,5222116,28.097672,34.501015,37.365983,2300291.0,276463.0,10150.0,2586905.0,88.920583,10.687018,0.392361,50447,47287,1.066826,2.7,4571,267895,1.706266,2.1,5310903.0,73.27,15.28,2279,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
3,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,thunderstorm,NaN,2550,NaN,68200.0,11.79,9.25,6.71,9.19,1851519,1941174,1993026,5787064,31.994099,33.543330,34.439329,2317336.0,278466.0,11010.0,2606813.0,88.895368,10.682239,0.422355,51598,48156,1.071476,0.6,5364,277627,1.932089,2.2,5380443.0,73.27,15.28,2279,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
4,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 00:00:00,02:00:00,2015-07-19 00:00:00,07:00:00,severe weather,NaN,NaN,1740,250,250000.0,13.07,10.16,7.74,10.43,2028875,2161612,1777937,5970339,33.982576,36.205850,29.779498,2374674.0,289044.0,9812.0,2673531.0,88.821637,10.811320,0.367005,54431,49844,1.092027,1.7,4873,292023,1.668704,2.2,5489594.0,73.27,15.28,2279,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2011.0,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06 00:00:00,08:00:00,2011-12-06 00:00:00,20:00:00,public appeal,NaN,NaN,720,155,34500.0,8.41,7.80,6.20,7.56,488853,438133,386693,1313678,37.212544,33.351628,29.435904,330738.0,60017.0,3639.0,394394.0,83.859795,15.217524,0.922681,57012,47586,1.198083,9.8,934,39067,2.390765,0.5,685326.0,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1530,2006.0,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaN,NaN,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,1650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,309997.0,53709.0,2331.0,366037.0,84.690072,14.673107,0.636821,42913,48909,0.877405,3.5,1019,27868,3.656524,0.7,649422.0,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1531,2009.0,8.0,South Dakota,SD,RFC,West North Central,

In [6]:
start_time = pd.to_datetime(
    df['OUTAGE.START.DATE'].dropna().astype(str).str.split(' ').str[0] 
    + ' ' 
    + df['OUTAGE.START.TIME'].dropna().astype(str)
)

restoration_time = pd.to_datetime(
    df['OUTAGE.RESTORATION.DATE'].dropna().astype(str).str.split(' ').str[0] 
    + ' ' 
    + df['OUTAGE.RESTORATION.TIME'].dropna().astype(str)
)

In [7]:
df['OUTAGE.RESTORATION'] = restoration_time
df['OUTAGE.START'] = start_time
df = df.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis = 1)
df

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.RESTORATION,OUTAGE.START
0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,NaN,NaN,3060,NaN,70000.0,11.60,9.18,6.81,9.28,2332915,2114774,2113291,6562520,35.549073,32.225029,32.202431,2308736.0,276286.0,10673.0,2595696.0,88.944776,10.644005,0.411181,51268,47586,1.077376,1.6,4802,274182,1.751391,2.2,5348119.0,73.27,15.28,2279,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2011-07-03 20:00:00,2011-07-01 17:00:00
1,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,vandalism,NaN,1,NaN,NaN,12.12,9.71,6.49,9.28,1586986,1807756,1887927,5284231,30.032487,34.210389,35.727564,2345860.0,284978.0,9898.0,2640737.0,88.833534,10.791609,0.374820,53499,49091,1.089792,1.9,5226,291955,1.790002,2.2,5457125.0,73.27,15.28,2279,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2014-05-11 18:39:00,2014-05-11 18:38:00
2,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,heavy wind,NaN,3000,NaN,70000.0,10.87,8.19,6.07,8.15,1467293,1801683,1951295,5222116,28.097672,34.501015,37.365983,2300291.0,276463.0,10150.0,2586905.0,88.920583,10.687018,0.392361,50447,47287,1.066826,2.7,4571,267895,1.706266,2.1,5310903.0,73.27,15.28,2279,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-10-28 22:00:00,2010-10-26 20:00:00
3,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,thunderstorm,NaN,2550,NaN,68200.0,11.79,9.25,6.71,9.19,1851519,1941174,1993026,5787064,31.994099,33.543330,34.439329,2317336.0,278466.0,11010.0,2606813.0,88.895368,10.682239,0.422355,51598,48156,1.071476,0.6,5364,277627,1.932089,2.2,5380443.0,73.27,15.28,2279,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2012-06-20 23:00:00,2012-06-19 04:30:00
4,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,NaN,NaN,1740,250,250000.0,13.07,10.16,7.74,10.43,2028875,2161612,1777937,5970339,33.982576,36.205850,29.779498,2374674.0,289044.0,9812.0,2673531.0,88.821637,10.811320,0.367005,54431,49844,1.092027,1.7,4873,292023,1.668704,2.2,5489594.0,73.27,15.28,2279,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2015-07-19 07:00:00,2015-07-18 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2011.0,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,public appeal,NaN,NaN,720,155,34500.0,8.41,7.80,6.20,7.56,488853,438133,386693,1313678,37.212544,33.351628,29.435904,330738.0,60017.0,3639.0,394394.0,83.859795,15.217524,0.922681,57012,47586,1.198083,9.8,934,39067,2.390765,0.5,685326.0,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,2011-12-06 20:00:00,2011-12-06 08:00:00
1530,2006.0,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,1650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,309997.0,53709.0,2331.0,366037.0,84.690072,14.673107,0.636821,42913,48909,0.877405,3.5,1019,27868,3.656524,0.7,649422.0,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaT,NaT
1531,2009.0,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,islanding,NaN,NaN,59,84,NaN,9.25,7.47,5.53,7.67,337874,370771,215406,924051,36.564432,40.124517,23.311051,367206.0,65971.0,3052.0,436229.0,84.177347,15.123020,0.699633

This part displays the number of occrences for each of the seven unique cause categories for power outages.

In [8]:
df['CAUSE.CATEGORY'].value_counts()

severe weather                   763
intentional attack               418
system operability disruption    127
public appeal                     69
equipment failure                 60
fuel supply emergency             51
islanding                         46
Name: CAUSE.CATEGORY, dtype: int64

### Baseline Model

In this section, we will implement our baseline model using 5 features: [`'YEAR'`, `'U.S._STATE'`, `'OUTAGE.DURATION'`, `'POPULATION'`, `'CUSTOMERS.AFFECTED'`] to predict `CAUSE.CATEGORY`.

In [9]:
# TODO
# Y: CUSTOMERS.AFFECTED
# X: YEAR, U.S._STATE, CAUSE.CATEGORY, DURATION, POPULATION

In [10]:
from sklearn.preprocessing import Binarizer

Out of the 55 columns, we only select those that we'll use.

In [11]:
ndf = df[['YEAR', 'U.S._STATE', 'CAUSE.CATEGORY', 'OUTAGE.DURATION', 'POPULATION', 'CUSTOMERS.AFFECTED', 'TOTAL.CUSTOMERS']]
ndf = ndf.dropna()
ndf

,YEAR,U.S._STATE,CAUSE.CATEGORY,OUTAGE.DURATION,POPULATION,CUSTOMERS.AFFECTED,TOTAL.CUSTOMERS
0,2011.0,Minnesota,severe weather,3060,5348119.0,70000.0,2595696.0
2,2010.0,Minnesota,severe weather,3000,5310903.0,70000.0,2586905.0
3,2012.0,Minnesota,severe weather,2550,5380443.0,68200.0,2606813.0
4,2015.0,Minnesota,severe weather,1740,5489594.0,250000.0,2673531.0
5,2010.0,Minnesota,severe weather,1860,5310903.0,60000.0,2586905.0
...,...,...,...,...,...,...,...
1522,2004.0,Idaho,system operability disruption,95,1391802.0,35000.0,701140.0
1523,2011.0,Idaho,intentional attack,360,1584134.0,0.0,794925.0
1524,2003.0,Idaho,public appeal,1548,1363380.0,0.0,687334.0
1526,2016.0,Idaho,intentional attack,0,1680026.0,0.0,849763.0


We first divide them into features and target variable, then, we splitted the training and testing dataset.

In [12]:
X = ndf[['YEAR', 'U.S._STATE', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'POPULATION']]
y = ndf['CAUSE.CATEGORY']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

This section transforms the catgeorical varaibles using OneHotEncoder and keeps the same values for numeric variables.

In [14]:
cat = ['YEAR', 'U.S._STATE']
num = ['OUTAGE.DURATION', 'CUSTOMERS.AFFECTED', 'POPULATION']

cat_func = OneHotEncoder(handle_unknown='ignore')
num_func = FunctionTransformer(lambda x:x)

ct = ColumnTransformer([('categorical', cat_func, cat), ('numerical', num_func, num)])

In [15]:
baseline_pl = Pipeline([('column', ct), ('dtc', (DecisionTreeClassifier(max_depth=15)))])

In [16]:
baseline_pl.fit(X_train, y_train)

Pipeline(steps=[('column',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['YEAR', 'U.S._STATE']),
                                                 ('numerical',
                                                  FunctionTransformer(func=<function <lambda> at 0x7f60e8acaa60>),
                                                  ['OUTAGE.DURATION',
                                                   'CUSTOMERS.AFFECTED',
                                                   'POPULATION'])])),
                ('dtc', DecisionTreeClassifier(max_depth=15))])

Accuracy is not a good measure, but we will display it anyway for our thought process.

In [17]:
baseline_pl.score(X_train, y_train)

1.0

In [18]:
baseline_pl.score(X_test, y_test)

0.8349056603773585

We observe that the diagonal entries of the confusion matrix indeed aligns with True Positives of the multi-label confusion matrix. Then, we defined a helper function `calculate_recall` to compute **recall** for each category, and finally calculate their mean as our evaluation metric.

In [19]:
# categories contains all causes
categories = ndf['CAUSE.CATEGORY'].unique()
categories

array(['severe weather', 'intentional attack', 'public appeal',
       'system operability disruption', 'islanding', 'equipment failure',
       'fuel supply emergency'], dtype=object)

In [20]:
base_pred = baseline_pl.predict(X_test)

In [21]:
metrics.confusion_matrix(y_test, base_pred,labels=categories)

array([[127,   1,   3,   8,   0,   5,   0],
       [  2,  38,   0,   1,   0,   1,   0],
       [  0,   1,   1,   0,   0,   0,   0],
       [  2,   1,   0,  10,   1,   0,   0],
       [  0,   2,   0,   0,   1,   1,   0],
       [  3,   0,   0,   2,   0,   0,   0],
       [  0,   0,   0,   1,   0,   0,   0]])

In [22]:
multi_matrix = metrics.multilabel_confusion_matrix(y_test, base_pred,labels=categories)
multi_matrix
# Counts of TN / FP / FN / TP

array([[[ 61,   7],
        [ 17, 127]],

       [[165,   5],
        [  4,  38]],

       [[207,   3],
        [  1,   1]],

       [[186,  12],
        [  4,  10]],

       [[207,   1],
        [  3,   1]],

       [[200,   7],
        [  5,   0]],

       [[211,   0],
        [  1,   0]]])

In [23]:
metrics.recall_score(y_test, base_pred, average="macro")

0.46442743764172334

We will further attempt feature engineering that could potentially improve our model and determine the performance comparing the average recall with that of our baseline model, and then determine our final model.

### Feature Engineering: Population Density & Standard Scaler

In this section, we will engineer two new features.
The first calculates the population density using:

$$
{\rm Population\ density} = \frac {POPPCT\_URBAN * POPDEN\_URBAN + (1 - POPPCT\_URBAN) * POPDEN\_RURAL}{100} 
$$

We think that it will be a very effective feature that could potentially increase our recall score, since population density can be an indicative measure for different causes. For example, major outages that occur within areas with higher population density might due to `system operability disruption`.

In addition, we'll also engineer new features by applying standard scaler to other numeric columns.

In [79]:
ndf = df[['YEAR', 'U.S._STATE', 'CAUSE.CATEGORY', 'OUTAGE.DURATION', 'POPULATION', 'CUSTOMERS.AFFECTED', 'POPPCT_URBAN', 'POPDEN_URBAN', 'POPDEN_RURAL']]
ndf = ndf.dropna()
ndf

,YEAR,U.S._STATE,CAUSE.CATEGORY,OUTAGE.DURATION,POPULATION,CUSTOMERS.AFFECTED,POPPCT_URBAN,POPDEN_URBAN,POPDEN_RURAL
0,2011.0,Minnesota,severe weather,3060,5348119.0,70000.0,73.27,2279,18.2
2,2010.0,Minnesota,severe weather,3000,5310903.0,70000.0,73.27,2279,18.2
3,2012.0,Minnesota,severe weather,2550,5380443.0,68200.0,73.27,2279,18.2
4,2015.0,Minnesota,severe weather,1740,5489594.0,250000.0,73.27,2279,18.2
5,2010.0,Minnesota,severe weather,1860,5310903.0,60000.0,73.27,2279,18.2
...,...,...,...,...,...,...,...,...,...
1522,2004.0,Idaho,system operability disruption,95,1391802.0,35000.0,70.58,2216.8,5.6
1523,2011.0,Idaho,intentional attack,360,1584134.0,0.0,70.58,2216.8,5.6
1524,2003.0,Idaho,public appeal,1548,1363380.0,0.0,70.58,2216.8,5.6
1526,2016.0,Idaho,intentional attack,0,1680026.0,0.0,70.58,2216.8,5.6


In [80]:
X = ndf[['YEAR', 'U.S._STATE', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'POPULATION', 'POPPCT_URBAN', 'POPDEN_URBAN', 'POPDEN_RURAL']]
y = ndf['CAUSE.CATEGORY']

In [83]:
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X, y, test_size=0.2)

We define a function that computes the population density for each obsevation.

In [84]:
def pop_den(df):
    # us area, unit: square kilometer
    den = (df['POPPCT_URBAN'] * df['POPDEN_URBAN'] + (1 - df['POPPCT_URBAN']) * df['POPDEN_RURAL']) / 100
    return den.to_frame()

Based on baseline model, we still apply `OneHotEncoder` for categorical variables `YEAR` and `U.S._STATE`. But for variables regarding population, we pass them into our function transformer `pop_den`, and other numeric variables to `StandardScaler`.

In [85]:
cat = ['YEAR', 'U.S._STATE']
num = ['OUTAGE.DURATION', 'CUSTOMERS.AFFECTED']
popden = ['POPPCT_URBAN', 'POPDEN_URBAN', 'POPDEN_RURAL']

cat_func = OneHotEncoder(handle_unknown='ignore')

num_func = StandardScaler()

pop_func = FunctionTransformer(pop_den)

ct = ColumnTransformer([('categorical', cat_func, cat), ('numerical', num_func, num), ('density', pop_func, popden)])

In [86]:
#cat_func = OneHotEncoder(handle_unknown='ignore')

#num_func = FunctionTransformer(lambda x: x * x)

#ct = ColumnTransformer([('categorical', cat_func, cat), ('numerical', num_func, num)])

We pass in the column transformer and DTC to our final pipeline (remain using max depth 15 until we perform grid search).

In [87]:
final_pl = Pipeline([('column', ct), ('dtc', (DecisionTreeClassifier(max_depth=15)))])

In [88]:
final_pl.fit(X_train_n, y_train_n)

Pipeline(steps=[('column',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['YEAR', 'U.S._STATE']),
                                                 ('numerical', StandardScaler(),
                                                  ['OUTAGE.DURATION',
                                                   'CUSTOMERS.AFFECTED']),
                                                 ('density',
                                                  FunctionTransformer(func=<function pop_den at 0x7f60e4aa8af0>),
                                                  ['POPPCT_URBAN',
                                                   'POPDEN_URBAN',
                                                   'POPDEN_RURAL'])])),
                ('dtc', DecisionTreeClassifier(max_depth=15))])

In [89]:
final_pred = final_pl.predict(X_test_n)

In [90]:
metrics.recall_score(y_test_n, final_pred, average = 'macro')

0.5929688232178038

We observed that we obtained a much higher recall score after engineered two new features. To further justify this, we will fit the baseline model again using this training/testing set and observe the recall score.

In [108]:
baseline_pl.fit(X_train_n, y_train_n)

Pipeline(steps=[('column',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['YEAR', 'U.S._STATE']),
                                                 ('numerical',
                                                  FunctionTransformer(func=<function <lambda> at 0x7f60e8acaa60>),
                                                  ['OUTAGE.DURATION',
                                                   'CUSTOMERS.AFFECTED',
                                                   'POPULATION'])])),
                ('dtc', DecisionTreeClassifier(max_depth=15))])

In [109]:
base_pred = baseline_pl.predict(X_test_n)

In [111]:
metrics.recall_score(y_test_n, base_pred, average = 'macro')

0.5351966436904938

Indeed, our final pipeline got improved from the basline model. However, to obtain the final model we still have to run grid search to get best parameters.

### Final Model

In [112]:
# on baseline
final_pl.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'column', 'dtc', 'column__n_jobs', 'column__remainder', 'column__sparse_threshold', 'column__transformer_weights', 'column__transformers', 'column__verbose', 'column__categorical', 'column__numerical', 'column__density', 'column__categorical__categories', 'column__categorical__drop', 'column__categorical__dtype', 'column__categorical__handle_unknown', 'column__categorical__sparse', 'column__numerical__copy', 'column__numerical__with_mean', 'column__numerical__with_std', 'column__density__accept_sparse', 'column__density__check_inverse', 'column__density__func', 'column__density__inv_kw_args', 'column__density__inverse_func', 'column__density__kw_args', 'column__density__validate', 'dtc__ccp_alpha', 'dtc__class_weight', 'dtc__criterion', 'dtc__max_depth', 'dtc__max_features', 'dtc__max_leaf_nodes', 'dtc__min_impurity_decrease', 'dtc__min_impurity_split', 'dtc__min_samples_leaf', 'dtc__min_samples_split', 'dtc__min_weight_fraction_leaf', 'dtc__ran

In [113]:
grid_params = {
    'dtc__max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 15, 20],
    'dtc__min_samples_split': [9, 11, 13, 15, 17, 20], 
    'dtc__min_samples_leaf': [1, 2, 3, 4, 5]
}

In [114]:
search = GridSearchCV(final_pl, grid_params, cv = 3)

In [115]:
search.fit(X_train_n, y_train_n)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('column',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['YEAR',
                                                                          'U.S._STATE']),
                                                                        ('numerical',
                                                                         StandardScaler(),
                                                                         ['OUTAGE.DURATION',
                                                                          'CUSTOMERS.AFFECTED']),
                                                                        ('density',
                                                                         FunctionTransformer(func=<functio

In [116]:
search.best_params_

{'dtc__max_depth': 5, 'dtc__min_samples_leaf': 2, 'dtc__min_samples_split': 17}

In [128]:
final_model = Pipeline([('column', ct), ('dtc', (DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, min_samples_split=17)))])

In [129]:
final_model.fit(X_train_n, y_train_n)

Pipeline(steps=[('column',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['YEAR', 'U.S._STATE']),
                                                 ('numerical', StandardScaler(),
                                                  ['OUTAGE.DURATION',
                                                   'CUSTOMERS.AFFECTED']),
                                                 ('density',
                                                  FunctionTransformer(func=<function pop_den at 0x7f60e4aa8af0>),
                                                  ['POPPCT_URBAN',
                                                   'POPDEN_URBAN',
                                                   'POPDEN_RURAL'])])),
                ('dtc',
                 DecisionTreeClassifier(max_depth=5, min_samples_leaf=2,
                                       

In [130]:
final_m_pred = final_model.predict(X_test_n)

In [131]:
metrics.recall_score(y_test_n, final_m_pred, average = 'macro')

0.4798664629251773

According to instruction, we will take the model as is. We could further improve the efficiency if we adjust Grid Search so that it would find the best parameter that optimizes recall score, currently it finds the best parameters for optimizing accuracy.

### Fairness Evaluation

To assess the fairness and bias of our model, we specifically chose the column `POPPCT_URBAN` to construct our X and Y subsets and hypothesized that the urban population percentage doesn't affect our recall scores.

We will binarize our datasets into:

`1`: which has `POPPCT_URBAN` higher than the median urban population percentage;

`0`: which are lower than the median.

Our test statistic would be the **absolute difference** between the **recall score** of these two subsets. 

***Null Hypothesis***: Urban population percentage, either higher or lower than its' median, doesn't have an effect on our recall score.(i.e. the difference between the recall scores between two subsets are due to chance)

***Alternative Hypothesis***: The urban population percentage does influence our recall score on two subsets.

In [170]:
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [171]:
last_df = ndf.copy()
last_df

,YEAR,U.S._STATE,CAUSE.CATEGORY,OUTAGE.DURATION,POPULATION,CUSTOMERS.AFFECTED,POPPCT_URBAN,POPDEN_URBAN,POPDEN_RURAL
0,2011.0,Minnesota,severe weather,3060,5348119.0,70000.0,73.27,2279,18.2
2,2010.0,Minnesota,severe weather,3000,5310903.0,70000.0,73.27,2279,18.2
3,2012.0,Minnesota,severe weather,2550,5380443.0,68200.0,73.27,2279,18.2
4,2015.0,Minnesota,severe weather,1740,5489594.0,250000.0,73.27,2279,18.2
5,2010.0,Minnesota,severe weather,1860,5310903.0,60000.0,73.27,2279,18.2
...,...,...,...,...,...,...,...,...,...
1522,2004.0,Idaho,system operability disruption,95,1391802.0,35000.0,70.58,2216.8,5.6
1523,2011.0,Idaho,intentional attack,360,1584134.0,0.0,70.58,2216.8,5.6
1524,2003.0,Idaho,public appeal,1548,1363380.0,0.0,70.58,2216.8,5.6
1526,2016.0,Idaho,intentional attack,0,1680026.0,0.0,70.58,2216.8,5.6


In [172]:
pop_med = np.median(last_df['POPPCT_URBAN'])
pop_med

84.05

In [173]:
# first binarize the dataframe
last_df['BINARIZED'] = (last_df['POPPCT_URBAN'] >= pop_med).astype(int)
last_df

,YEAR,U.S._STATE,CAUSE.CATEGORY,OUTAGE.DURATION,POPULATION,CUSTOMERS.AFFECTED,POPPCT_URBAN,POPDEN_URBAN,POPDEN_RURAL,BINARIZED
0,2011.0,Minnesota,severe weather,3060,5348119.0,70000.0,73.27,2279,18.2,0
2,2010.0,Minnesota,severe weather,3000,5310903.0,70000.0,73.27,2279,18.2,0
3,2012.0,Minnesota,severe weather,2550,5380443.0,68200.0,73.27,2279,18.2,0
4,2015.0,Minnesota,severe weather,1740,5489594.0,250000.0,73.27,2279,18.2,0
5,2010.0,Minnesota,severe weather,1860,5310903.0,60000.0,73.27,2279,18.2,0
...,...,...,...,...,...,...,...,...,...,...
1522,2004.0,Idaho,system operability disruption,95,1391802.0,35000.0,70.58,2216.8,5.6,0
1523,2011.0,Idaho,intentional attack,360,1584134.0,0.0,70.58,2216.8,5.6,0
1524,2003.0,Idaho,public appeal,1548,1363380.0,0.0,70.58,2216.8,5.6,0
1526,2016.0,Idaho,intentional attack,0,1680026.0,0.0,70.58,2216.8,5.6,0


In [174]:
set1 = last_df[last_df['BINARIZED'] == 1]
set2 = last_df[last_df['BINARIZED'] == 0]

In [175]:
X_1 = set1[['YEAR', 'U.S._STATE', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'POPULATION', 'POPPCT_URBAN', 'POPDEN_URBAN', 'POPDEN_RURAL']]
y_1 = set1['CAUSE.CATEGORY']
X_2 = set2[['YEAR', 'U.S._STATE', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'POPULATION', 'POPPCT_URBAN', 'POPDEN_URBAN', 'POPDEN_RURAL']]
y_2 = set2['CAUSE.CATEGORY']

In [176]:
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, test_size=0.2)
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2, test_size=0.2)

In [177]:
model_1 = final_model.fit(X_1_train, y_1_train)
model_2 = final_model.fit(X_2_train, y_2_train)

In [178]:
recall_1 = metrics.recall_score(y_1_test, model_1.predict(X_1_test), average = 'macro')
recall_2 = metrics.recall_score(y_2_test, model_2.predict(X_2_test), average = 'macro')
obs_stat = abs(recall_1 - recall_2)
obs_stat

0.03629304300036007

In [179]:
N = 1000

test_stats = []

for _ in range(N):
    cur_df = last_df.copy()
    cur_df['POPPCT_URBAN'] = np.random.permutation(cur_df['POPPCT_URBAN'])
    cur_df['BINARIZED'] = (cur_df['POPPCT_URBAN'] >= pop_med).astype(int)
    cur1 = cur_df[cur_df['BINARIZED'] == 1]
    cur2 = cur_df[cur_df['BINARIZED'] == 0]
    
    X_1 = cur1[['YEAR', 'U.S._STATE', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'POPULATION', 'POPPCT_URBAN', 'POPDEN_URBAN', 'POPDEN_RURAL']]
    y_1 = cur1['CAUSE.CATEGORY']
    X_2 = cur2[['YEAR', 'U.S._STATE', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'POPULATION', 'POPPCT_URBAN', 'POPDEN_URBAN', 'POPDEN_RURAL']]
    y_2 = cur2['CAUSE.CATEGORY']
    
    X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(X_1, y_1, test_size=0.2)
    X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2, test_size=0.2)
    
    model_1 = final_model.fit(X_1_train, y_1_train)
    model_2 = final_model.fit(X_2_train, y_2_train)
    
    recall_1 = metrics.recall_score(y_1_test, model_1.predict(X_1_test), average = 'macro')
    recall_2 = metrics.recall_score(y_2_test, model_2.predict(X_2_test), average = 'macro')
    test_stat = abs(recall_1 - recall_2)
    
    test_stats.append(test_stat)
    
test_stats

[0.05147196460478182,
 0.13494208494208498,
 0.033737218424106985,
 0.10257592640321855,
 0.059326479971641255,
 0.06882230829599262,
 0.008862734637788694,
 0.06088435374149659,
 0.03080206830206833,
 0.27639116970603944,
 0.1836291951917906,
 0.10416666666666669,
 0.19393530389943653,
 0.17019337417241148,
 0.12422661994090561,
 0.084997403094418,
 0.06453749615788346,
 0.09224178481394102,
 0.11605737947352857,
 0.19564857656962925,
 0.08182847192257148,
 0.0933461686917717,
 0.1414632768840205,
 0.05349212998982589,
 0.1404268246373509,
 0.17353868353868357,
 0.1405467436654559,
 0.08872960372960376,
 0.05181453634085209,
 0.027436527436527414,
 0.019257585862287874,
 0.2609346347234547,
 0.21477190876350544,
 0.030154927118791974,
 0.0892021720969089,
 0.1629888428801473,
 0.048978286966960716,
 0.08472222222222225,
 0.04276346902397332,
 0.07560596310596318,
 0.15056519624515485,
 0.016136560660942045,
 0.058675375132562935,
 0.14161146739223912,
 0.08344799908267397,
 0.01466272

In [182]:
p_val = (test_stats >= obs_stat).mean()
p_val

0.762

Pick a significance level 0.05, this is much higher than what we expected, hence rejecting the null hypothesis. Favoring the alternative, the population percentage does influence our recall score on two datasets. It's not a cogent illustration of a perfectly fair model.